In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import os
import yaml

# Load the YAML file
def load_api_key(yml_file):
    with open(yml_file, 'r') as file:
        config = yaml.safe_load(file)  # Safely load the YAML file
        return config.get('openai_key')  # Retrieve the 'api_key'

# Usage

os.environ['OPENAI_API_KEY'] =load_api_key('chatgpt_api_credentials.yml')

In [4]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')
prompt_agent

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature= 0)

llm.invoke('Hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-b031553a-219a-48fb-bc2e-c0f3f681b12e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Agent = LLM + Tools

In [6]:
import subprocess
from langchain_core.tools import tool

@tool
def list_files():
    """
    List files in the current directory.
    """
    try:
        # Check if the current directory exists
        current_dir = os.getcwd()
        items = os.listdir(current_dir)

        # Separate files and folders
        files = [f for f in items if os.path.isfile(os.path.join(current_dir, f))]
        folders = [f for f in items if os.path.isdir(os.path.join(current_dir, f))]

        # Format the response
        response = ""
        if folders:
            response += "Folders:\n" + "\n".join(folders) + "\n\n"
        if files:
            response += "Files:\n" + "\n".join(files) + "\n"

        if not response:
            response = "The current directory is empty."

        return response
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [7]:
list_files.invoke({"input":""})



'Folders:\n.git\n\nFiles:\n.gitignore\n1.0_Intro_To_Langchain.ipynb\n1.1_Hands_on_simple_chains.ipynb\n1.2_LCEL_and_runnables.ipynb\n1.3_langchain_simple_agent.ipynb\nchatgpt_api_credentials.yml\nLICENSE\nREADME.md\n'

In [8]:
llm_with_tool = llm.bind_tools([list_files])

In [9]:
llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001BABA5ED100>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001BABA5FC2B0>, root_client=<openai.OpenAI object at 0x000001BAB6719E20>, root_async_client=<openai.AsyncOpenAI object at 0x000001BABA5ED160>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'list_files', 'description': 'List files in the current directory.', 'parameters': {'properties': {}, 'type': 'object'}}}]}, config={}, config_factories=[])

In [10]:
dictionary_with_tool_names ={
    "list_files":list_files
}

output = llm_with_tool.invoke("List files in current directory")

if output.tool_calls:
    for tool_call in output.tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        tool_func = dictionary_with_tool_names[tool_name]
        print(tool_func.invoke(tool_args))

Folders:
.git

Files:
.gitignore
1.0_Intro_To_Langchain.ipynb
1.1_Hands_on_simple_chains.ipynb
1.2_LCEL_and_runnables.ipynb
1.3_langchain_simple_agent.ipynb
chatgpt_api_credentials.yml
LICENSE
README.md



**Agent = LLM + Tools**

In [11]:
from langchain.agents import create_tool_calling_agent, AgentExecutor


In [12]:
tools = [list_files]

agent = create_tool_calling_agent(llm, tools, prompt_agent)

agent_executor = AgentExecutor(agent =agent, tools = tools, verbose=True)

In [13]:
agent_executor.invoke({'input': 'List files in current directory'})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


Folders:
.git

Files:
.gitignore
1.0_Intro_To_Langchain.ipynb
1.1_Hands_on_simple_chains.ipynb
1.2_LCEL_and_runnables.ipynb
1.3_langchain_simple_agent.ipynb
chatgpt_api_credentials.yml
LICENSE
README.md
Here are the files in the current directory:

### Folders:
- .git

### Files:
- .gitignore
- 1.0_Intro_To_Langchain.ipynb
- 1.1_Hands_on_simple_chains.ipynb
- 1.2_LCEL_and_runnables.ipynb
- 1.3_langchain_simple_agent.ipynb
- chatgpt_api_credentials.yml
- LICENSE
- README.md

> Finished chain.


{'input': 'List files in current directory',
 'output': 'Here are the files in the current directory:\n\n### Folders:\n- .git\n\n### Files:\n- .gitignore\n- 1.0_Intro_To_Langchain.ipynb\n- 1.1_Hands_on_simple_chains.ipynb\n- 1.2_LCEL_and_runnables.ipynb\n- 1.3_langchain_simple_agent.ipynb\n- chatgpt_api_credentials.yml\n- LICENSE\n- README.md'}

In [14]:
agent_executor.invoke({'input': 'What kind of files are in current directory?'})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


Folders:
.git

Files:
.gitignore
1.0_Intro_To_Langchain.ipynb
1.1_Hands_on_simple_chains.ipynb
1.2_LCEL_and_runnables.ipynb
1.3_langchain_simple_agent.ipynb
chatgpt_api_credentials.yml
LICENSE
README.md
In the current directory, there are the following files and folders:

### Folders:
- `.git`

### Files:
- `.gitignore`
- `1.0_Intro_To_Langchain.ipynb`
- `1.1_Hands_on_simple_chains.ipynb`
- `1.2_LCEL_and_runnables.ipynb`
- `1.3_langchain_simple_agent.ipynb`
- `chatgpt_api_credentials.yml`
- `LICENSE`
- `README.md`

> Finished chain.


{'input': 'What kind of files are in current directory?',
 'output': 'In the current directory, there are the following files and folders:\n\n### Folders:\n- `.git`\n\n### Files:\n- `.gitignore`\n- `1.0_Intro_To_Langchain.ipynb`\n- `1.1_Hands_on_simple_chains.ipynb`\n- `1.2_LCEL_and_runnables.ipynb`\n- `1.3_langchain_simple_agent.ipynb`\n- `chatgpt_api_credentials.yml`\n- `LICENSE`\n- `README.md`'}

#### TAVILY SEARCH AGENT

In [15]:
# !pip install langchain-community

In [16]:
from langchain_community.tools import TavilySearchResults

In [ ]:
import os
import yaml

# Load the YAML file
def load_api_key(yml_file):
    with open(yml_file, 'r') as file:
        config = yaml.safe_load(file)  # Safely load the YAML file
        return config.get('tavily_key')  # Retrieve the 'api_key'

# Usage

os.environ['TAVILY_API_KEY'] =load_api_key('tavily_api_key.yml')